# Mental Health Service Analysis - Exploratory Data Analysis

This notebook provides an initial exploration of mental health service utilization data for UNT students.

## Objectives
1. Load and inspect the data
2. Understand data distributions
3. Identify patterns and trends
4. Detect potential service gaps


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")

## 1. Load Data

In [ ]:
# Load the dataset
df = pd.read_csv('../data/raw/mental_health_data.csv')

# Convert date column
df['appointment_date'] = pd.to_datetime(df['appointment_date'])

print(f"Dataset loaded: {len(df):,} records")
print(f"Date range: {df['appointment_date'].min()} to {df['appointment_date'].max()}")
print(f"Unique students: {df['student_id'].nunique():,}")

In [ ]:
# Display first few rows
df.head(10)

In [ ]:
# Data info
df.info()

## 2. Data Quality Assessment

In [ ]:
# Check for missing values
missing_data = df.isnull().sum()
missing_pct = (missing_data / len(df)) * 100

missing_summary = pd.DataFrame({
    'Missing Count': missing_data,
    'Percentage': missing_pct
})

print("Missing Values Summary:")
print(missing_summary[missing_summary['Missing Count'] > 0])

In [ ]:
# Check for duplicates
duplicates = df.duplicated(subset=['student_id', 'appointment_date', 'service_type']).sum()
print(f"Duplicate records: {duplicates}")

In [ ]:
# Summary statistics
df.describe()

## 3. Service Utilization Analysis

In [ ]:
# Service type distribution
plt.figure(figsize=(12, 6))
service_counts = df['service_type'].value_counts()
sns.barplot(x=service_counts.values, y=service_counts.index, palette='viridis')
plt.title('Service Type Distribution', fontsize=16, fontweight='bold')
plt.xlabel('Number of Appointments')
plt.ylabel('Service Type')
plt.tight_layout()
plt.show()

print("\nService Type Percentages:")
print((service_counts / service_counts.sum() * 100).round(2))

In [ ]:
# Utilization over time
df['year_month'] = df['appointment_date'].dt.to_period('M')
monthly_counts = df.groupby('year_month').size()

plt.figure(figsize=(14, 6))
monthly_counts.plot(kind='line', marker='o', linewidth=2)
plt.title('Monthly Service Utilization', fontsize=16, fontweight='bold')
plt.xlabel('Month')
plt.ylabel('Number of Appointments')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Demographic Analysis

In [ ]:
# Student year distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# By student year
year_counts = df['student_year'].value_counts()
axes[0].pie(year_counts.values, labels=year_counts.index, autopct='%1.1f%%', startangle=90)
axes[0].set_title('Distribution by Student Year')

# By college
college_counts = df['student_college'].value_counts()
axes[1].barh(range(len(college_counts)), college_counts.values)
axes[1].set_yticks(range(len(college_counts)))
axes[1].set_yticklabels([c.replace('College of ', '') for c in college_counts.index])
axes[1].set_title('Distribution by College')
axes[1].set_xlabel('Number of Appointments')

plt.tight_layout()
plt.show()

In [ ]:
# International and first-generation students
print("International Students:")
print(df['international_student'].value_counts(normalize=True) * 100)
print("\nFirst Generation Students:")
print(df['first_generation'].value_counts(normalize=True) * 100)

## 5. Wait Time Analysis

In [ ]:
# Wait time statistics
print("Wait Time Statistics:")
print(df['wait_days'].describe())
print(f"\nMedian wait time: {df['wait_days'].median():.1f} days")
print(f"Students waiting >7 days: {(df['wait_days'] > 7).sum() / len(df) * 100:.1f}%")
print(f"Students waiting >14 days: {(df['wait_days'] > 14).sum() / len(df) * 100:.1f}%")

In [ ]:
# Wait time distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(df['wait_days'].dropna(), bins=30, edgecolor='black', alpha=0.7)
axes[0].axvline(df['wait_days'].mean(), color='red', linestyle='--', label=f'Mean: {df["wait_days"].mean():.1f}')
axes[0].axvline(df['wait_days'].median(), color='green', linestyle='--', label=f'Median: {df["wait_days"].median():.1f}')
axes[0].set_title('Wait Time Distribution')
axes[0].set_xlabel('Days')
axes[0].set_ylabel('Frequency')
axes[0].legend()

# Box plot by service type
df.boxplot(column='wait_days', by='service_type', ax=axes[1], rot=45)
axes[1].set_title('Wait Time by Service Type')
axes[1].set_xlabel('Service Type')
axes[1].set_ylabel('Wait Days')

plt.tight_layout()
plt.show()

## 6. Service Gaps Identification

In [ ]:
# Average wait time by college and service type
gap_analysis = df.groupby(['student_college', 'service_type'])['wait_days'].agg(['mean', 'count']).reset_index()
gap_analysis = gap_analysis[gap_analysis['count'] > 10]  # Filter for significance
gap_analysis = gap_analysis.sort_values('mean', ascending=False)

print("Top 10 Service Gaps (Highest Wait Times):")
print(gap_analysis.head(10))

In [ ]:
# Heatmap of wait times
pivot_data = df.pivot_table(
    values='wait_days',
    index='student_college',
    columns='service_type',
    aggfunc='mean'
)

plt.figure(figsize=(12, 8))
sns.heatmap(pivot_data, annot=True, fmt='.1f', cmap='RdYlGn_r', cbar_kws={'label': 'Avg Wait Days'})
plt.title('Average Wait Time Heatmap\n(by College and Service Type)', fontsize=14, fontweight='bold')
plt.xlabel('Service Type')
plt.ylabel('College')
plt.tight_layout()
plt.show()

## 7. Key Findings Summary

In [ ]:
print("=" * 80)
print("KEY FINDINGS")
print("=" * 80)

print(f"\n1. UTILIZATION METRICS")
print(f"   - Total students served: {df['student_id'].nunique():,}")
print(f"   - Total appointments: {len(df):,}")
print(f"   - Average appointments per student: {len(df) / df['student_id'].nunique():.1f}")

print(f"\n2. WAIT TIME INSIGHTS")
print(f"   - Average wait time: {df['wait_days'].mean():.1f} days")
print(f"   - Percentage with extended wait (>7 days): {(df['wait_days'] > 7).sum() / len(df) * 100:.1f}%")

print(f"\n3. SERVICE DISTRIBUTION")
top_services = df['service_type'].value_counts().head(3)
for service, count in top_services.items():
    print(f"   - {service}: {count:,} ({count/len(df)*100:.1f}%)")

print(f"\n4. NO-SHOW ANALYSIS")
print(f"   - Overall no-show rate: {df['no_show'].mean() * 100:.1f}%")
print(f"   - Correlation with wait time: {df['wait_days'].corr(df['no_show'].astype(int)):.3f}")

print(f"\n5. DEMOGRAPHIC INSIGHTS")
print(f"   - International student usage: {df['international_student'].sum() / len(df) * 100:.1f}%")
print(f"   - First-generation student usage: {df['first_generation'].sum() / len(df) * 100:.1f}%")

print("\n" + "=" * 80)